<a href="https://colab.research.google.com/github/marianecosta-de/marianecosta-de/blob/main/script_inferencia_pet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Informações básicas
Script para enviar dados de Inferência Pet para MParticle -> Braze.
- O código itera sobre cada unique_id (ID único) encontrado nos dados.
Para cada `unique_id` ele filtra o `json_array` para encontrar todas as linhas que possuem o mesmo ID.
- Em seguida, para cada linha correspondente, a função map_dict é aplicada para obter um dicionário filtrado.
Os dicionários filtrados são adicionados a uma lista chamada requests_data, que é usada para construir a "carga útil" (todos os resultados relacionados ao mesmo public_id - mesmo que em linhas diferentes -) das requisições.

- Em resumo, o script lê os dados de um CSV, transforma e manipula um JSON e por fim, na mesma requisição (para os dados não sobrescreverem) envia todas as informações atreladas ao public_id.

https://petlove.atlassian.net/wiki/spaces/MS/pages/765198337/Infer+ncia+Pet

In [17]:
import pandas as pd
import json
from google.colab import drive
#drive.mount('/content/drive')

In [24]:
df = pd.read_csv("", sep=',') #adicionar nome do arquivo .csv
json_array = (json.loads(df.to_json(orient='records')))

endpoint = 'https://rest.iad-06.braze.com/users/track' #endpoint
api_key = 'ec7b57cf-1a89-4810-b822-7589c8c258d8' #api_key de prod
api_key_stag = 'fe69fb06-f5be-4d1f-88d2-38209aff6092'

retry = 0
contador = 1
total_public_ids = df["public_id"].count()

In [25]:
import time
import requests


retry_attempts = 3
retry_delay = 300  # 5 minutes in seconds
field_length = 255  # Maximum number of characters in each field

json_array = json.loads(df.to_json(orient='records'))
unique_ids = list(set([ client["public_id"] for client in json_array ]))

def map_dict(line):
  return {
    "pet_species": line["pet_species"],
    "pet_port": line["pet_port"],
    "pet_age": line["pet_age"],
    "pet_castrated": line["pet_castrated"]
  }

requests_data = []

for unique_id in unique_ids:
  # Get every line that matches the current unique_id
  matches = [ map_dict(line) for line in json_array if line["public_id"] == unique_id ]

  requests_data.append({
      "attributes": [{
        "external_id": unique_id,
        "pet_infered_test": matches
    }]
  })

In [ ]:
for payload in requests_data:

  json_payload = json.dumps(payload)
  headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key_stag}'
  }

  for attempt in range(1, retry_attempts + 1):
    try:
      with requests.post(endpoint, data=json_payload, headers=headers) as response:
          response.raise_for_status()
      print(f"Upload efetuado {contador}/{total_public_ids}")
      contador += 1
      break
    except requests.exceptions.RequestException as err:
      if attempt < retry_attempts:
          print(f"Erro encontrado, tentando novamente em {retry_delay} segundos...")
          print(f"Data enviado: {json_payload}")
          time.sleep(retry_delay)
      else:
          print("Máximo de tentativas atingido. Desistindo.")
          print(f"Data enviado: {json_payload}")
          print(f"Erro: {err}")